### Morocco World Cup 2022 squad

In [ ]:
from IPython.display import Image
Image(filename='squad_list.jpg')

In [2]:
#imports here
import face_recognition
import cv2 as cv
import csv
import numpy as np
from datetime import datetime
import pandas as pd

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time

### Step 1: squad list using WebScraping

In [4]:
url="https://www.fourfourtwo.com/features/morocco-world-cup-2022-squad-vahid-halilhodzic-names-his-latest-squad"

In [5]:
#Disable Alerts/Notifications
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)

In [ ]:
#specify the path to chromedriver.exe
driver = webdriver.Chrome('chromedriver.exe', chrome_options=chrome_options)
#open the webpage
squad=[]
driver.get(url=url)
for i in range(1,27):
    l=WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="article-body"]/ul/li[{i}]')))
    squad.append(l.text)

In [ ]:
squad[0]

In [ ]:
squad

In [36]:
len(squad)

26

In [37]:
squad=pd.Series(squad)

In [38]:
squad=squad.apply(lambda s : s.split(': ')[1].split(' (')[0])

In [ ]:
squad

In [40]:
squad=list(squad)

In [ ]:
squad

In [42]:
with open('players_2022.txt', 'w') as f:
    for line in squad[:-1] :
        f.write(line)
        f.write('\n')
    f.write(squad[-1])

### Step 2: how to manipulate face_recognition

In [ ]:
#image = face_recognition.load_image_file("squad_list.jpg")
#face_locations = face_recognition.face_locations(image)
#face_locations

In [ ]:
#train image
Image(filename='WalidRegragui.jpg')

In [ ]:
#test image
Image(filename='image.jpg')

In [45]:
# Load the jpg file into a numpy array
known_image = face_recognition.load_image_file("WalidRegragui.jpg")
unknown_image = face_recognition.load_image_file("image.jpg")

regragi_encoding = face_recognition.face_encodings(known_image)[0]
unknown_encoding = face_recognition.face_encodings(unknown_image)[0]

results = face_recognition.compare_faces([regragi_encoding], unknown_encoding)

In [ ]:
if results[0]==True:
    print('they are the same : walid regragi')
else:
    print('Not walid regragi')

In [47]:
from PIL import Image
# Load the jpg file into a numpy array
image = face_recognition.load_image_file("rm.jpg")

# Find all the faces in the image using the default HOG-based model.
# This method is fairly accurate, but not as accurate as the CNN model and not GPU accelerated.
face_locations = face_recognition.face_locations(image)

print("I found {} face(s) in this photograph.".format(len(face_locations)))

for face_location in face_locations:

    # Print the location of each face in this image
    top, right, bottom, left = face_location
    print("A face is located at pixel location Top: {}, Left: {}, Bottom: {}, Right: {}".format(top, left, bottom, right))

    # You can access the actual face itself like this:
    face_image = image[top:bottom, left:right]
    pil_image = Image.fromarray(face_image)
    pil_image.show()

I found 2 face(s) in this photograph.
A face is located at pixel location Top: 17, Left: 59, Bottom: 60, Right: 103
A face is located at pixel location Top: 22, Left: 199, Bottom: 65, Right: 242


### Step 3: Extract and Process the Photos


In [48]:
with open('players_2022.txt') as f:
    lines = f.readlines()

In [50]:
lim=int(input("Entrer le nombre d'image pour chaque joueur pour entrainer le modele"))

In [ ]:
from bing_image_downloader import downloader
for a in lines:
    t=a.split('\n')[0]    
    downloader.download(t, limit=lim,  output_dir='dataset', 
    adult_filter_off=True, force_replace=False, timeout=60)

In [ ]:
known=[]
names=[]

In [ ]:
len(squad)

In [ ]:
lines

In [ ]:
len(lines)

In [ ]:
for i in range(26):
    print(i)
    a=lines[i].split('\n')[0]
    print(a)
    for j in range(lim):
        m=j+1
        img=face_recognition.load_image_file('dataset(test)/'+str(a)+'/Image_{}.jpg'.format(m))
        k=face_recognition.face_encodings(img)[0]
        known.append(k)
        names.append(a)

In [ ]:
known

In [ ]:
len(known)

In [ ]:
#with open('players_encoding.txt', 'w') as f:
#    i=0
#    for k,i in zip(known,range(26)):
#        f.write('\n')
#        f.write(f'{lines[i]}'+'\n')
#        f.write(str(k))
#        f.write('\n')

In [ ]:
p=[]
for i in range(26):
    a=0
    p+=[str(k) for k in known[a:a+lim]]
    a+=lim
with open('encoding_multiple_matrix.txt', 'w') as f:
    i=0
    for k,i in zip(p,range(26)):
        f.write('\n')
        f.write(f'{lines[i]}'+'\n')
        f.write(str(k))
        f.write('\n')

In [ ]:
players=names.copy()
face_loc=[]
face_enc=[]
face_names=[]
s=True

In [ ]:
now=datetime.now()
current_date=now.strftime("%Y-%m-%d")

In [ ]:
f=open(current_date+'.csv','w+',newline='')
lnwriter=csv.writer(f)

In [ ]:
cascade_classifier = cv.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')

In [ ]:
Vid=cv.VideoCapture(0)

In [ ]:
while True:
    _,frame=Vid.read()
    s_frame=cv.resize(frame,(0,0),fx=0.25,fy=0.25)
    s_frame2=s_frame[:,:,::-1]
    gray = cv.cvtColor(frame, 0)
    detections = cascade_classifier.detectMultiScale(gray,scaleFactor=1.3,minNeighbors=5)
    if(len(detections) > 0):
        (x,y,w,h) = detections[0]
        frame = cv.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
    if s:
        face_loc=face_recognition.face_locations(s_frame2)
        face_enc=face_recognition.face_encodings(s_frame2,face_loc)
        face_names=[]
        for face_en in face_enc:
            m=face_recognition.compare_faces(known,face_en)
            name=""
            face_dist=face_recognition.face_distance(known,face_en)
            best_match_index=np.argmin(face_dist)
            if m[best_match_index]:
                name=names[best_match_index]
            face_names.append(name)
            if name in names:
                if name in players:
                    players.remove(name)
                    print(players)
                    current_time=now.strftime("%H-%M-%S")
                    lnwriter.writerow([name,current_time])
    cv.imshow("players world cup 2022",frame)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
Vid.release()
cv.destroyAllWindows()
f.close()